In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'C:\\Users\\tsbau\\git\\idp-radio-1'

In [2]:
import os 
import tensorflow as tf
from pathlib import Path

In [3]:
# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:08:00.0, compute capability: 7.5



In [3]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, generate_benchmarks, METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

In [ ]:
#CHEXPERT_BENCHMARKS, CHESTXRAY14_BENCHMARKS = generate_benchmarks ()

In [ ]:
CHEXPERT_BENCHMARKS.keys() 

In [ ]:
CHESTXRAY14_BENCHMARKS.keys()

In [ ]:
#[m.name for m in METRICS]

In [ ]:
#CHEXPERT_COLUMNS

In [8]:
reduced_columns = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']
len(reduced_columns)

12

In [6]:
!remote_access/get_tunnels.sh

URLs open for the following services:
[('jupyterlab', 'http://4ee29cc6b5ce.ngrok.io'), ('tensorboard', 'http://4cded4af3852.ngrok.io')]


In [ ]:
def simple_architecture_experiment(benchmark, base_model_fn, classes ):
    model = SimpleBaseArchitecture(base_model_fn, len(classes))
    experiment = Experiment(benchmark, model)
    return experiment

In [ ]:
chexpert_resnet_exp = simple_architecture_experiment(CHEXPERT_BENCHMARKS["WBCE_E20_B32"], DenseNet121, CHEXPERT_COLUMNS)


In [ ]:
import random
random.seed()

In [ ]:
chexpert_resnet_exp.run()

In [ ]:
print(chexpert_resnet_exp.evaluation_result["report"])

In [ ]:
chexpert_resnet_exp.model_name

In [ ]:
chestxray14_resnet_exp = simple_architecture_experiment(CHESTXRAY14_BENCHMARKS["WBCE_E20_B32"], DenseNet121, CHESTXRAY14_COLUMNS)

In [ ]:
print(chestxray14_resnet_exp.model_description )


In [ ]:
chestxray14_resnet_exp.run()

In [ ]:
predictions = experiment_inceptionnet_chexpert.model.predict(
            experiment_inceptionnet_chexpert.benchmark.testgen, steps=len(experiment_inceptionnet_chexpert.benchmark.testgen), verbose=1)

In [ ]:
true_labels = experiment_inceptionnet_chexpert.benchmark.testgen.get_encoded_labels()

In [ ]:
# model_guendel_chestxray14 = densenet(classes=len(CHESTXRAY14_COLUMNS))
model_guendel_chexpert = densenet(classes=len(CHEXPERT_COLUMNS))

#experiment_guendel_chestxray14 = Experiment(chestxray14_benchmark, model_guendel_chestxray14)
experiment_guendel_chexpert = Experiment(CHEXPERT_BENCHMARKS["WBCE_E10_B32"], model_guendel_chexpert, model_name="test_WBCE_32")



In [ ]:
#experiment_guendel_chestxray14_result =  experiment_guendel_chestxray14.run()

In [ ]:
experiment_guendel_chexpert_result =  experiment_guendel_chexpert.run()

In [ ]:
experiment_guendel_chexpert.evaluate()

In [ ]:
experiment_guendel_chexpert.save(upload=False)

In [ ]:
print("done")

In [ ]:
CHEXPERT_BENCHMARKS["WBCE_E10_B32"].as_dict()

In [26]:
y_true

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.,  1., nan,  1.],
       [ 0.,  0.,  1.,  0.]], dtype=float32)>

In [82]:
weights_pos = tf.ones((4)) * 20.0
weights_neg = tf.ones((4)) * 0.5
y_true = tf.constant([[0, 1, float("nan"), 1], [float("nan"), 0, 1, float("nan")]], dtype=tf.float32)
y_pred = tf.constant([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]])
loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
loss

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([nan, nan], dtype=float32)>

In [83]:
mask = tf.math.is_nan(y_true)
mask

<tf.Tensor: shape=(2, 4), dtype=bool, numpy=
array([[False, False,  True, False],
       [ True, False, False,  True]])>

In [41]:
tf.boolean_mask(y_pred, mask)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.3, 0.5, 0.8], dtype=float32)>

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.        , -1.6094373 ,         nan, -0.9162905 ],
       [ 0.69314694,  0.        , -0.35667482,  0.2231434 ]],
      dtype=float32)>

In [46]:
from tensorflow.keras import backend as K
from tensorflow.python.ops.losses import util as tf_losses_util
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import constant_op
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.keras import backend_config

In [92]:
epsilon = backend_config.epsilon
if tensor_util.is_tensor(y_pred) and tensor_util.is_tensor(y_true):
    y_pred, y_true = tf_losses_util.squeeze_or_expand_dimensions(
        y_pred, y_true)

epsilon_ = constant_op.constant(
    epsilon(), dtype=y_pred.dtype.base_dtype)
y_pred = clip_ops.clip_by_value(y_pred, epsilon_, 1. - epsilon_)

# Compute cross entropy from probabilities.
bce_pos = y_true * math_ops.log(y_pred + epsilon_)
bce_neg = (1 - y_true) * math_ops.log(1 - y_pred + epsilon_)
bce = bce_pos * weights_pos +  bce_neg * weights_neg
bce

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ -0.0526802, -32.188747 ,         nan, -18.32581  ],
       [        nan,  -0.4581453,  -7.1334963,         nan]],
      dtype=float32)>

In [93]:
tf.reduce_mean(tf.boolean_mask(-bce, tf.math.is_finite(bce)))

<tf.Tensor: shape=(), dtype=float32, numpy=11.631776>

In [60]:
y_true * math_ops.log(y_pred + epsilon_)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[-0.        , -1.6094373 ,  1.2039725 , -0.9162905 ],
       [ 0.69314694, -0.        , -0.35667482,  0.2231434 ]],
      dtype=float32)>

In [90]:
tf.boolean_mask(-bce, tf.math.is_finite(bce))

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([ 0.0526802, 32.188747 , 18.32581  ,  0.4581453,  7.1334963],
      dtype=float32)>

In [100]:
bench = Chexpert_Benchmark("Chexpert_CWBCE",
                           epochs=12,
                           batch_size=1,
                           metrics=METRICS,
                           crop=True, 
                           nan_replacement = float("nan"))
bench.loss = WeightedBinaryCrossentropy(bench.positive_weights,
                       bench.negative_weights)

ZeroDivisionError: division by zero

In [ ]:
bench.traingen[0]